The tutorial that is followed in this notebook is found [here](https://www.tensorflow.org/versions/r0.11/tutorials/recurrent/index.html).

In [13]:
import tensorflow as tf
import numpy as np
from tensorflow.models.rnn.ptb import reader

flags = tf.flags
flags.DEFINE_string("data_path", None, #"./simple-examples/data", 
                    "Where the training-test data is stored.")
flags.DEFINE_string("save_path", "./model",
                   "Model output directory.")
flags.DEFINE_bool("use_fp16", False,
                 "Train using 16-bit floats instead of 32bit floats.")

FLAGS = flags.FLAGS

In [2]:
def data_type():
    return tf.float16 if FLAGS.use_fp16 else tf.float32

class PTBInput(object):
  """The input data."""
  def __init__(self, config, data, name=None):
    self.batch_size = batch_size = config.batch_size
    self.num_steps = num_steps = config.num_steps
    self.epoch_size = ((len(data) // batch_size) - 1) // num_steps
    self.input_data, self.targets = reader.ptb_producer(
        data, batch_size, num_steps, name=name)

In [3]:
class PTBModel(object):
    """The PTB model."""
    def __init__(self, is_training, config, input_):
        self._input = input_
        
        batch_size = input_.batch_size
        num_steps = input_.num_steps
        size = config.hidden_size
        vocab_size = config.vocab_size
        
        # Here is the implementation of the LSTM cells
        lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(
            size, 
            forget_bias=0.0, 
            state_is_tuple=True)
        if is_training and config.keep_prob < 1:
            lstm_cell = tf.nn.rnn_cell.DropoutWrapper(
                lstm_cell, output_keep_prob=config.keep_prob)
        cell = tf.nn.rnn_cell.MultiRNNCell(
            [lstm_cell] * config.num_layers, 
            state_is_tuple=True
        )
        
        self._initial_state = cell.zero_state(batch_size, data_type())
        
        with tf.device("/cpu:0"):
            embedding = tf.get_variable(
                "embedding",
                [vocab_size, size],
                dtype=data_type()
            )
            inputs = tf.nn.embedding_lookup(embedding, input_.input_data)
            
        if is_training and config.keep_prob < 1:
            inputs = tf.nn.dropout(inputs, config.keep_prob)

        # This code below is for tutorial purposes,
        # if you want to have the optimized version use these lines:
        # inputs = [tf.squeeze(input_step, [1])
        #    for input_step in tf.split(1, num_steps, inputs)]
        # outputs, state = tf.nn.rnn(
        #    cell, inputs, initial_state=self._initial_state)
        outputs = []
        state = self._initial_state
        
        with tf.variable_scope("RNN"):
            for time_step in range(num_steps):
                if time_step > 0: 
                    tf.get_variable_scope().reuse_variables()
                (cell_output, state) = cell(inputs[:, time_step, :], state)
                outputs.append(cell_output)
        
        output = tf.reshape(tf.concat(1, outputs), [-1, size])
        softmax_w = tf.get_variable(
            "softmax_w", [size, vocab_size], dtype=data_type()
        )
        softmax_b = tf.get_variable(
            "softmax_b", [vocab_size], dtype=data_type()
        )
        logits = tf.matmul(output, softmax_w) + softmax_b
        loss = tf.nn.seq2seq.sequence_loss_by_example(
            [logits],
            [tf.reshape(input_.targets, [-1])],
            [tf.ones([batch_size * num_steps], dtype=data_type())]
        )
        self._cost = cost = tf.reduce_sum(loss) / batch_size
        self._final_state = state
        
        if not is_training:
            return
        
        self._lr = tf.Variable(0.0, trainable=False)
        tvars = tf.trainable_variables()
        grad, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars),
                                        config.max_grad_norm)
        optimizer = tf.train.GradientDescentOptimizer(self._lr)
        self._train_op = optimizer.apply_gradients(
            zip(grads, tvars),
            global_step=tf.contrib.framework.get_or_create_global_step()
        )
        
        self._new_lr = tf.placeholder(
            tf.float32, shape=[], name="new_learning_rate"
        )
        self._lr_update = tf.assign(self._lr, self._new_lr)
        
    def assign_lr(self, session, lr_value):
        session.run(self._lr_update, feed_dict={self._new_lr: lr_value})
        
    @property
    def input(self):
        return self._input

    @property
    def initial_state(self):
        return self._initial_state

    @property
    def cost(self):
        return self._cost

    @property
    def final_state(self):
        return self._final_state

    @property
    def lr(self):
        return self._lr

    @property
    def train_op(self):
        return self._train_op

In [4]:
class SmallConfig(object):
    """Small config."""
    init_scale = 0.1
    learning_rate = 1.0
    max_grad_norm = 5
    num_layers = 2
    num_steps = 20
    hidden_size = 200
    max_epoch = 4
    max_max_epoch = 13
    keep_prob = 1.0
    lr_decay = 0.5
    batch_size = 20
    vocab_size = 10000
    
class TestConfig(object):
    """Tiny config, for testing."""
    init_scale = 0.1
    learning_rate = 1.0
    max_grad_norm = 1
    num_layers = 1
    num_steps = 2
    hidden_size = 2
    max_epoch = 1
    max_max_epoch = 1
    keep_prob = 1.0
    lr_decay = 0.5
    batch_size = 20
    vocab_size = 10000

In [5]:
def run_epoch(session, model, eval_op=None, verbose=False):
    start_time = time.time()
    costs = 0.0
    iters = 0
    state = session.run(model.initial_state)
    
    fetches = {
        "cost": model.cost,
        "final_state": model.final_state
    }
    if eval_op is not None:
        fetches["eval_op"] = eval_op
        
    for step in range(model.input.epoch_size):
        feed_dict = {}
        for i, (c, h) in enumerate(model.initial_state):
            feed_dict[c] = state[i].c
            feed_dict[h] = state[i].h
        
        vals = session.run(fetches, feed_dict)
        cost = vals["cost"]
        state = vals["final_state"]
        
        costs += cost
        iters += model.input.num_steps
        
        if verbose and step % (model.input.epoch_size // 10) == 10:
            print("%.3f perplexity: %.3f speed: %.0f wps" %
                 (step * 1.0 / model.input.epoch_size, np.exp(costs / iters),
                 iters * model.input.batch_size / (time.time() - start_time))
            )
    
    return np.exp(costs / iters)

In [6]:
# Main 
raw_data = reader.ptb_raw_data(FLAGS.data_path)
train_data, valid_data, test_data, _ = raw_data

config = TestConfig()
eval_config = TestConfig()
eval_config.batch_size = 1
eval_config.num_steps = 1

with tf.Graph().as_default():
    initializer = tf.random_uniform_initializer(
        -config.init_scale,
        config.init_scale
    )
    with tf.name_scope("Train"):
        train_input = PTBInput(
            config=config, data=train_data, name="TrainInput")
        
        with tf.variable_scope("Model", reuse=None, initializer=initializer):
            m = PTBModel(is_training=True, config=config, input_=train_input)
        tf.scalar_summary("Training Loss", m.cost)
        tf.scalar_summary("Learning Rate", m.lr)

TypeError: join() argument must be str or bytes, not 'NoneType'